In [ ]:
import pandas as pd
from datetime import datetime
import _beowFmt as fm
import _beowDDE as bwdde
import numpy as np

# 設置顯示選項
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 880)  # 設置顯示寬度
# pd.set_option('display.max_columns', None)  # 顯示所有列
pd.set_option('display.max_rows', None)  # 顯示所有行
pd.set_option('display.expand_frame_repr', False)  # 不自動換行


def process_dde_data(columns, data, sep):
    # 打印原始數據以檢查格式
    # print("Raw DDE data:")
    # for item in data:
    #     print(item) 

    processed_data = []
    for item in data:
        # 如果數據是單個字符串，嘗試分割
        if isinstance(item, str):
            # 這裡使用製表符（\t）分割，您可能需要根據實際格式調整
            processed_item = item.split(sep)
            if len(processed_item) != len(columns):
                print(len(processed_item),len(columns))
                print(f"Warning: Unexpected number of fields in item: {item}")
                continue
        else:
            print(f"Warning: Unexpected data type for item: {type(item)}")
            continue
        processed_data.append(processed_item)
    
    return pd.DataFrame(processed_data, columns=columns)


# 假設 today 是今天的日期
# today = pd.to_datetime('2023-10-01')

# 定義一個函數來處理日期替換和轉換
def process_date(date_str):
    if date_str == '--':
        date_str = '2022/1/1'
    return pd.to_datetime(date_str, format='%Y/%m/%d', errors='coerce')

# 定義一個函數來檢查日期並轉換為字符串格式
def check_date(date, today):
    if pd.isna(date):
        return ''
    if date.date() > today:
        return date.strftime('%Y/%m/%d')
    else:
        return ''


def Main(topic, ddeDict, stock_Ids, sep):
    service = "XQLITE"

    dde_List = list(ddeDict.keys())   #將dictionary物件轉換成list欄位
    print(dde_List)
    ddeItems = [f"{code}.TW-{",".join(dde_List)}" for code in stock_Ids]
    data = bwdde.fetch_multiple_dde_data(service, topic, ddeItems)
    # print(data)
    today = datetime.now().date()
    
    if data:
        df = process_dde_data(dde_List, data, sep)
        df = df.rename(columns=ddeDict)  # 重命名列名

        # 篩選出日期大於今天的行，否則設為空字串
        # today = pd.to_datetime(today)
        # df['除息日'] = df['除息日'].str.replace('--','2022/1/1')
        # df['除息日'] = pd.to_datetime(df['除息日'], format='%Y/%m/%d')
        # df['除息日'] = df['除息日'].apply(lambda x: x if x.date() > today else '')
        # df['除息日'] = df['除息日'].apply(lambda x: x.strftime('%Y/%m/%d') if isinstance(x, pd.Timestamp) else '')

        # df['法說會日'] = df['法說會日'].str.replace('--','2022/1/1')
        # df['法說會日'] = pd.to_datetime(df['法說會日'], format='%Y/%m/%d')
        # df['法說會日'] = df['法說會日'].apply(lambda x: x if x.date() > today else '')
        # df['法說會日'] = df['法說會日'].apply(lambda x: x.strftime('%Y/%m/%d') if isinstance(x, pd.Timestamp) else '')

        # 將 '--' 替換為 '2022/1/1'，然後轉換為 datetime 格式
        df['除息日'] = df['除息日'].apply(process_date)
        df['除息日'] = df['除息日'].apply(lambda x: check_date(x, today))
        df['法說會日'] = df['法說會日'].apply(process_date)
        df['法說會日'] = df['法說會日'].apply(lambda x: check_date(x, today)) # 使用 apply 來檢查日期，並將結果轉換為字符串格式
        df['股東會日'] = df['股東會日'].apply(process_date)
        df['股東會日'] = df['股東會日'].apply(lambda x: check_date(x, today))

        df['公司動態'] = df['公司動態'].str.replace('--', '')
        str2Float_columns = ['股本', 'PE', 'PB', 'ROE%', '當月營收YOY', '當月營收MOM', '稅後淨利成長率%']
        df[str2Float_columns] = df[str2Float_columns].replace('--', '0').astype(float).round(1) 
        # print(df)

        csv_file = r'webJson\\stock_finance.csv'
        df.to_csv(csv_file, index=False)   #這邊產生CSV檔提供給下面的去合併
    else:
        print("Failed to fetch data")


# 追蹤股票號碼
stock_Ids = open(r'stock\\sorted.txt', "r").read().split(",")
dde_fin_dict = {
            'ID': '代碼',
            'Capital': '股本',
            'ROE': 'ROE%',
            'PERatio': 'PE', 
            'PBRatio': 'PB',  
            'MonthlyNetSalesYoY': '當月營收YOY',
            'TFXRatio':            '當月營收MOM',
            'NetIncomeGrowthRate': '稅後淨利成長率%',
            'ChipsField25':    '除息日', #dvd ex-dividend date
            'CompanyNews':  '公司動態',
            'DateofMeetingofShareHolders': '股東會日',
            'RevenueMonth': '營收月份',
            'InvestorConferenceDate':'法說會日',
        }

Main("Quote", dde_fin_dict, stock_Ids, ";")


['ID', 'Capital', 'ROE', 'PERatio', 'PBRatio', 'MonthlyNetSalesYoY', 'TFXRatio', 'NetIncomeGrowthRate', 'ChipsField25', 'CompanyNews', 'DateofMeetingofShareHolders', 'RevenueMonth', 'InvestorConferenceDate']
